## Imports

In [1]:
from langchain_ollama import ChatOllama
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda, RunnableParallel
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain_core.documents import Document
from os.path import join, dirname, realpath

import re
import dotenv
import os
import pandas as pd


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from os.path import join, dirname, realpath

import re
import dotenv
import os
import pandas as pd


## Variables

In [2]:
MAIN_PATH = realpath(join(dirname("__file__"), '.'))

dotenv.load_dotenv(join(MAIN_PATH, ".env"));

In [3]:
tiles_csv_path = join(MAIN_PATH, "tiles_analyzer/", "tiles_description.csv")
tiles_filtered_csv_path = join(MAIN_PATH, join("./tiles_description.csv"))
tiles_vector_db_path = join(MAIN_PATH, "./chroma_tiles_db")
df_tiles = pd.read_csv(tiles_csv_path)

In [4]:
df_tiles_filtered = df_tiles.copy()
df_tiles_filtered["description"] = df_tiles["description"].apply(lambda row : row.replace("\"",""))
df_tiles_filtered["description"] = df_tiles_filtered["description"].apply(lambda row : re.sub(r'\s{2,}', ' ', row))
df_tiles_filtered["description"] = df_tiles_filtered["description"].apply(lambda row : row.replace("{ interpretations: [ ", ""))
df_tiles_filtered["description"] = df_tiles_filtered["description"].apply(lambda row : row.replace(" ]\n}", ""))
df_tiles_filtered["description"] = df_tiles_filtered["description"].apply(lambda row : "This can be " + row)

In [5]:
df_tiles_filtered.to_csv(join(MAIN_PATH, "tiles_description.csv"), index=False)

## Load Models

In [6]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = os.environ.get("GOOGLE_API_KEY")

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-preview-05-20",
)

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004"
)

## CreateDB

In [7]:
is_vector_database_created = os.path.exists(tiles_vector_db_path)

vector_store = None

if not is_vector_database_created:
    documents = []
    ids = []

    for i, row in df_tiles_filtered.iterrows():
        document = Document(
            page_content= row["description"],
            id=str(i),
            metadata={
                "b64image": row["base64"],
                "x": row["x"],
                "y": row["y"],
            }
        )
        ids.append(str(i))
        documents.append(document)

    vector_store = Chroma( 
        collection_name="Tiles_Descriptions",
        persist_directory=tiles_vector_db_path,
        embedding_function=embeddings
    )
    print(len(documents))
    
    vector_store.add_documents(documents=documents, ids=ids)
else:
    vector_store = Chroma( 
        collection_name="Tiles_Descriptions",
        persist_directory=tiles_vector_db_path,
        embedding_function=embeddings
    )

C:\Users\gurge\AppData\Local\Temp\ipykernel_20784\1593749702.py:22: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


1078


In [15]:
query = "A wizzard figure"

retriver = vector_store.as_retriever(
    search_kwargs={"k": 4}
)

relevant_docs = retriver.invoke(query)

for i, doc in enumerate(relevant_docs, 1):
    print(f"[Document {i}]")
    print(doc.page_content)
    print(f"({doc.metadata["x"]}, {doc.metadata["y"]})")

[Document 1]
This can be a hooded figure, a small golem, a dwarf character, a stone statue, a shopkeeper icon
(30, 3)
[Document 2]
This can be a ghostly apparition, a small spirit, a hooded figure, a cursed soul, a magic user icon
(26, 6)
[Document 3]
This can be a small ghost, a hooded figure, a spirit with a halo, a cursed doll, a magic user icon
(26, 7)
[Document 4]
This can be a hooded figure, a small shrine, a stone golem, a dark wizard, a mysterious altar
(26, 1)
